In [ ]:
import pandas as pd
!pip install plotnine
from plotnine import  ggplot, aes, geom_point, geom_line, labs, theme_minimal, facet_wrap, theme
import statsmodels.api as sm
import numpy as np



In [141]:
df = pd.read_csv("demographics.csv")
print(df.columns)

Index(['County', 'GEOID', 'anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'density', 'hisp', 'hispvap', 'nanglo',
       'nanglovap', 'official_boundary', 'pct_dem_lead', 'vap', 'votes_dem',
       'votes_rep', 'votes_total', 'state', 'total', 'area_sqmi',
       'college_enroll', 'bachelors_up_total_sum', '20_trump', '20_biden',
       '20_total', 'pct_dem_lead_20', 'households', 'aggregateincome',
       'anglovap_pct', 'asianvap_pct', 'hispvap_pct', 'blackvap_pct',
       'college_enroll_pct', 'bachelors_up_total_sum_pct', 'avg_hh_income',
       'shift_20_to_24'],
      dtype='object')


In [142]:
df_counties = df.groupby("County", as_index=False)[['anglo', 'anglovap', 'asian', 'asianvap', 'bh',
       'bhvap', 'black', 'blackvap', 'hisp', 'hispvap', 'nanglo',
       'nanglovap','vap', 'votes_dem','votes_rep', 'votes_total', 'college_enroll', 
       'bachelors_up_total_sum', '20_trump', '20_biden',
       'aggregateincome', '20_total', 'total', 'households'
       ]].sum()

df_counties['anglovap_pct'] = df_counties['anglovap'] / df_counties['vap']
df_counties['asianvap_pct'] = df_counties['asianvap'] / df_counties['vap']
df_counties['hispvap_pct'] = df_counties['hispvap'] / df_counties['vap']
df_counties['blackvap_pct'] = df_counties['blackvap'] / df_counties['vap']
df_counties['college_enroll_pct'] = df_counties['college_enroll'] / df_counties['vap']
df_counties['bachelors_up_total_sum_pct'] = df_counties['bachelors_up_total_sum'] / df_counties['vap']
df_counties['pct_dem_lead'] = df_counties['votes_dem'] / df_counties['votes_total'] - df_counties['votes_rep'] / df_counties['votes_total']
df_counties['pct_dem_lead_20'] = df_counties['20_biden'] / df_counties['20_total'] - df_counties['20_trump'] / df_counties['20_total']
df_counties['avg_hh_income'] = df_counties['aggregateincome'] / df_counties['households']
df_counties["swing_2024"] = df_counties["pct_dem_lead"] - df_counties["pct_dem_lead_20"]
print(df_counties["avg_hh_income"].sum())
# density
df_counties.to_csv("county_demographics.csv", index=False)

21943677.879743718


## poc counties x shift


college ed x shift

In [145]:
rel_df = pd.read_csv("demographics.csv")
pct_2020_dem = rel_df["20_biden"].sum() / rel_df["20_total"].sum()
pct_2020_rep = rel_df["20_trump"].sum() / rel_df["20_total"].sum()
pct_2020_margin = pct_2020_dem - pct_2020_rep
pct_2024_dem = rel_df["votes_dem"].sum() / rel_df["votes_total"].sum()
pct_2024_rep = rel_df["votes_rep"].sum() / rel_df["votes_total"].sum()
pct_2024_margin = pct_2024_dem - pct_2024_rep
pct_2024_margin_change = pct_2024_margin - pct_2020_margin
print(pct_2024_margin_change)
rel_df["relative_shift"] = rel_df["shift_20_to_24"] - pct_2024_margin_change
print(rel_df["relative_shift"].isna().sum())
rel_df.to_csv("demographics.csv", index=False)




-0.08062126509529405
213


In [147]:
print(rel_df["relative_shift"].describe())

count    9505.000000
mean        0.007097
std         0.138109
min        -1.533959
25%        -0.044379
50%         0.028451
75%         0.072661
max         2.080621
Name: relative_shift, dtype: float64


In [150]:
gains = rel_df[rel_df["shift_20_to_24"] > 0.01]
print(gains["avg_hh_income"].describe())

count      1513.000000
mean     118048.679183
std       51997.339808
min           0.000000
25%       82672.503092
50%      107580.184508
75%      141671.399177
max      421132.203390
Name: avg_hh_income, dtype: float64
